base.csv - анонимизированный набор товаров. Каждый товар представлен как уникальный id (0-base, 1-base, 2-base) и вектор признаков размерностью 72.

train.csv - обучающий датасет. Каждая строчка - один товар, для которого известен уникальный id (0-query, 1-query, …) , вектор признаков И id товара из base.csv, который максимально похож на него (по мнению экспертов).

validation.csv - датасет с товарами (уникальный id и вектор признаков), для которых надо найти наиболее близкие товары из base.csv

validation_answer.csv - правильные ответы к предыдущему файлу.

In [1]:
import pandas as pd
import numpy as np
import faiss
import zipfile
import os
import itertools
import requests
import io

In [13]:
import requests
import zipfile
import io
import pandas as pd

# Ссылка на ZIP-файл
url = 'https://s49vlx.storage.yandex.net/rdisk/8a10c08950ef073e9018020100b4bd72ff43c98e330c13fcd74ecadc5730c255/664ff442/gE7SLUtkG98n0esaTvJyH1WPRzedt-hn5czM70r4O8d2Q2ztdhI_e3q7mCYaD2sk6-DdtCiMuWdkR5Xp89JVLA==?uid=0&filename=data_mini.zip&disposition=attachment&hash=FMQKgeVS7dk36PddS8U4OmgkXkjvWSQ7PjfgUCRfYzmn/E91Ubrvwj/LVQFOahBEq/J6bpmRyOJonT3VoXnDag%3D%3D&limit=0&content_type=application%2Fzip&owner_uid=454825120&fsize=109746533&hid=880a26cf809b6d774a08cc0adc8536d4&media_type=compressed&tknv=v2&ts=6192980d41480&s=9547f135c881e90b1582db254f61061550812a5c24491cdc7808c4cf3f27fff4&pb=U2FsdGVkX1-7qKhhRoAQq6nj-t5ONjtnwZN5zDjdfdbitTJTSPq6LpM_xQm7jNMky6iIXUJqgGOBlH2XN5JP6NJAP2bJQfDsQnBNHbhUR28'

# Загружаем файл
response = requests.get(url)

# Открываем ZIP-архив
with zipfile.ZipFile(io.BytesIO(response.content), 'r') as zip_ref:
    # Выводим список файлов в архиве
    print("Список файлов в архиве:", zip_ref.namelist())
    
    # Читаем файлы непосредственно из архива и устанавливаем 'Id' в качестве индекса
    with zip_ref.open('base.csv') as base_file:
        base = pd.read_csv(base_file, index_col='Id')
    with zip_ref.open('train.csv') as train_file:
        train = pd.read_csv(train_file, index_col='Id')
    with zip_ref.open('validation.csv') as validation_file:
        validation = pd.read_csv(validation_file, index_col='Id')
    with zip_ref.open('validation_answer.csv') as validation_answer_file:
        validation_answer = pd.read_csv(validation_answer_file, index_col='Id')

# Теперь переменные base, train, validation и validation_answer содержат данные из соответствующих CSV-файлов с 'Id' в качестве индекса


Список файлов в архиве: ['base.csv', 'train.csv', 'validation.csv', 'validation_answer.csv']


In [3]:
# Предварительный анализ данных
for df, name in zip([base, train, validation, validation_answer], ['Base', 'Train', 'Validation', 'Validation Answer']):
    print(f"--- {name} Dataset ---")
    print("Первые 5 строк:")
    display(df.head())
    print("\nОсновная информация:")
    df.info()
    print("\nСтатистическая информация:")
    display(df.describe())
    print("\n")

--- Base Dataset ---
Первые 5 строк:


,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
Id,,,,,,,,,,,,,,,,,,,,,
4207931-base,-43.946243,15.364378,17.515854,-132.31146,157.064420,-4.069252,-340.630860,-57.550140,128.398220,45.090958,...,-71.927170,30.711966,-90.190475,-24.931271,66.972534,106.346634,-44.270622,155.98834,-1074.464888,-25.066608
2710972-base,-73.004890,4.923342,-19.750746,-136.52908,99.907170,-70.709110,-567.401996,-128.890150,109.914986,201.472200,...,-109.044660,20.916021,-171.201390,-110.596844,67.730100,8.909615,-9.470253,133.29536,-545.897014,-72.913230
1371460-base,-85.565570,-0.493598,-48.374817,-157.98502,96.809510,-81.710210,-22.297688,79.768670,124.357086,105.715180,...,-58.821650,41.369606,-132.934500,-43.016839,67.871925,141.778240,69.048520,111.72038,-1111.038833,-23.087206
3438601-base,-105.564090,15.393871,-46.223934,-158.11488,79.514114,-48.944480,-93.713010,38.581398,123.397960,110.324326,...,-87.907290,-58.806870,-147.794800,-155.830237,68.974754,21.397510,126.098785,139.73320,-1282.707248,-74.527940
422798-base,-74.638880,11.315012,-40.204174,-161.76430,50.507114,-80.775560,-640.923467,65.225000,122.344940,191.465850,...,-30.002094,53.642930,-149.823230,176.921371,69.473280,-43.395180,-58.947716,133.84064,-1074.464888,-1.164146



Основная информация:
<class 'pandas.core.frame.DataFrame'>
Index: 291813 entries, 4207931-base to 274130-base
Data columns (total 72 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   0       291813 non-null  float64
 1   1       291813 non-null  float64
 2   2       291813 non-null  float64
 3   3       291813 non-null  float64
 4   4       291813 non-null  float64
 5   5       291813 non-null  float64
 6   6       291813 non-null  float64
 7   7       291813 non-null  float64
 8   8       291813 non-null  float64
 9   9       291813 non-null  float64
 10  10      291813 non-null  float64
 11  11      291813 non-null  float64
 12  12      291813 non-null  float64
 13  13      291813 non-null  float64
 14  14      291813 non-null  float64
 15  15      291813 non-null  float64
 16  16      291813 non-null  float64
 17  17      291813 non-null  float64
 18  18      291813 non-null  float64
 19  19      291813 non-null  float64
 20  20      291813 

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
count,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,...,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000,291813.000000
mean,-86.274741,8.078087,-44.613480,-146.605552,111.261183,-71.875015,-393.430460,20.391975,123.676692,124.427158,...,-79.075207,33.244836,-154.720293,13.699277,67.792659,23.543147,74.890028,115.611366,-798.355219,-47.701336
std,24.918947,4.949495,38.545928,19.842726,46.350083,28.189743,272.084921,64.297194,6.352832,64.366631,...,30.485074,28.896144,41.217568,98.996416,1.825081,55.353032,61.320347,21.219222,385.414338,41.731025
min,-189.356020,-12.594500,-231.785920,-224.880500,-95.240830,-188.473330,-791.468770,-296.171050,93.684616,-143.499600,...,-214.821140,-85.825500,-346.239320,-157.593866,59.835790,-213.492420,-190.483150,18.601448,-1297.924962,-209.935760
25%,-103.154300,4.709408,-69.535900,-159.882740,80.367640,-91.227936,-631.937855,-22.085905,119.473625,81.685610,...,-98.823830,16.952824,-180.695560,-71.763964,66.582790,-12.501141,33.784820,101.675660,-1074.464888,-75.621110
50%,-86.308130,8.040010,-43.838493,-146.717360,111.776870,-71.751820,-424.306925,20.850153,123.882500,123.456270,...,-78.538180,34.682540,-153.902830,13.167371,67.815050,23.461630,74.876980,116.049270,-1074.464888,-48.403816
75%,-69.254616,11.466815,-19.694052,-133.320140,142.364610,-52.310825,-157.342700,63.989326,127.967766,167.014130,...,-58.638412,52.097710,-127.221360,99.362213,69.028800,59.740337,115.896880,129.625950,-504.291183,-19.684170
max,15.574616,28.751917,151.377080,-55.167892,299.307920,49.880882,109.631986,301.094240,147.870910,402.994600,...,48.822197,141.025270,17.344528,185.096719,75.474625,264.347200,319.601040,213.577260,98.770811,126.973220




--- Train Dataset ---
Первые 5 строк:


...

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,...,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,-85.307362,7.633750,-43.686537,-146.038791,111.209942,-72.767940,-440.587163,21.834861,123.051767,124.662559,...,-81.250167,36.956902,-152.122730,14.886141,67.791824,22.857614,72.524486,115.420796,-709.333484,-49.022258
std,25.911681,4.950078,38.805284,20.543489,47.779448,28.617899,278.953671,66.326518,6.522882,65.458934,...,30.617505,25.251097,40.945219,99.384247,1.851120,55.658492,62.595136,21.490729,405.917480,41.159872
min,-167.550670,-10.583933,-205.797360,-219.720890,-76.422760,-170.868380,-791.443909,-277.983950,97.360535,-118.700714,...,-189.231300,-47.206474,-308.200380,-157.580677,61.086390,-174.799100,-159.840730,27.746750,-1297.689518,-209.935760
25%,-103.253996,4.245802,-69.606605,-159.972975,78.592808,-92.371089,-739.549115,-22.112910,118.623887,81.167942,...,-101.935440,19.950575,-178.986322,-71.481630,66.545755,-14.034864,31.120448,100.431221,-1074.464888,-76.834515
50%,-85.495230,7.604729,-43.152172,-146.100195,111.398595,-72.734217,-512.600719,22.596903,123.105532,124.588370,...,-80.640330,36.343388,-151.159155,16.498258,67.809390,22.756214,71.640040,115.465535,-809.193143,-49.420510
75%,-67.228041,10.950509,-17.883776,-131.973093,143.645965,-53.415469,-204.531443,67.179237,127.583330,168.550240,...,-60.179952,53.856120,-124.446027,100.077341,69.043971,60.188393,114.197410,129.996610,-358.197327,-20.995152
max,10.861183,27.944784,107.040690,-73.909584,287.397200,47.001106,109.404851,251.090180,146.034410,350.088320,...,31.880241,141.025270,-17.406784,184.987267,73.886790,241.695620,289.583160,190.171420,98.747128,102.473755




--- Validation Answer Dataset ---
Первые 5 строк:


,Expected
Id,
196680-query,1087368-base
134615-query,849674-base
82675-query,4183486-base
162076-query,2879258-base
23069-query,615229-base



Основная информация:
<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 196680-query to 43566-query
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Expected  10000 non-null  object
dtypes: object(1)
memory usage: 156.2+ KB

Статистическая информация:


,Expected
count,10000
unique,9640
top,935520-base
freq,3


In [4]:
# Проверка на дубликаты и их удаление
for df, name in zip([base, train], ['Base', 'Train']):
    print(f"--- {name} Dataset ---")
    if df.duplicated().any():
        print("Обнаружены дубликаты!")
        print("Количество дубликатов до удаления:", df.duplicated().sum())
        df.drop_duplicates(inplace=True)
        print("Дубликаты удалены.")
        print("Количество дубликатов после удаления:", df.duplicated().sum())
    else:
        print("Дубликаты не обнаружены.")
    print("\n")


--- Base Dataset ---
Обнаружены дубликаты!
Количество дубликатов до удаления: 11278
Дубликаты удалены.
Количество дубликатов после удаления: 0


--- Train Dataset ---
Обнаружены дубликаты!
Количество дубликатов до удаления: 243
Дубликаты удалены.
Количество дубликатов после удаления: 0




In [5]:
# Фиксируем столбец "Target" в переменную targets
targets = train["Target"]

# Удаляем столбец "Target" из тренировочного набора данных
train.drop("Target", axis=1, inplace=True)

# Определяем размерность данных на основе количества столбцов в наборе данных base
dims = base.shape[1]

# Задаем количество кандидатов для поиска
candidate_number = 5

# Задаем значения для перебора
n_cells_values = [10, 20, 30, 50, 100, 200]  # Разные значения для количества ячеек
quantizer_types = [faiss.IndexFlatL2, faiss.IndexFlatIP]  # Разные типы квантизаторов
index_types = [faiss.IndexIVFFlat]  # Тип индексов

# Создаем комбинации параметров для перебора
parameter_combinations = list(itertools.product(n_cells_values, quantizer_types, index_types))

# Инициализируем переменные для лучшего результата и соответствующих параметров
best_accuracy = 0
best_parameters = None

# Проходим по всем комбинациям параметров
for n_cells, quantizer_type, index_type in parameter_combinations:
    # Создаем квантизатор
    quantizer = quantizer_type(dims)
    
    # Создаем индекс
    idx = index_type(quantizer, dims, n_cells)
    
    # Обучаем индекс
    idx.train(np.ascontiguousarray(base.values).astype('float32'))
    idx.add(np.ascontiguousarray(base.values).astype('float32'))
    
    # Выполняем поиск ближайших соседей
    r, idx_nearest = idx.search(np.ascontiguousarray(train.values).astype('float32'), candidate_number)
    
    # Создаем словарь для сопоставления индексов данных с их первоначальными индексами из DataFrame base
    base_index = {k: v for k, v in enumerate(base.index.to_list())}
    
    # Вычисляем точность
    accuracy = sum(int(target in [base_index[r] for r in el]) for target, el in zip(targets.values.tolist(), idx_nearest.tolist())) / len(idx_nearest)
    
    # Сравниваем с лучшим результатом
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_parameters = (n_cells, quantizer_type, index_type)

# Выводим лучший результат и соответствующие параметры
print(f"Лучший результат: точность = {best_accuracy:.1%}, параметры: n_cells = {best_parameters[0]}, quantizer_type = {best_parameters[1].__name__}, index_type = {best_parameters[2].__name__}")
